In [1]:
import sys
import os

current_dir = os.getcwd()
src_dir = os.path.join(current_dir, '..', 'src')
sys.path.append(src_dir)

import pandas as pd
from dqc.dqc import DataQualityPipeline
from etl.kaggle_data_loader import download_data

In [2]:
download_data()

data_paths = {}

import os
for dirname, _, filenames in os.walk('./etl/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data_paths[filename] = f"{dirname}/{filename}"

relations = {}

for file, path in data_paths.items():
    relations[file.split('.')[0]] = pd.read_csv(path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sandra/Projects/ds_practice/notebooks/etl/kaggle.json'